In [1]:
from proj1_helpers import load_csv_data, predict_labels, create_csv_submission
from created_helpers import *
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import pandas as pd
#import implementation as imp
import numpy as np
from datetime import datetime

In [2]:
def accuracy(y_pred, y_test):
    correct = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_test[i]:
            correct += 1
    return correct / len(y_test)

def standardize(x):
    """Standardize the original data set."""
    mean_x = np.mean(x, axis=0)
    x = x - mean_x
    std_x = np.std(x, axis=0)
    x = x / std_x
    return x, mean_x, std_x

In [4]:
data_path = './'
train_data_path = os.path.join(data_path, "train.csv")
test_data_path = os.path.join(data_path,"test.csv")

y_train, x_train, ids_train = load_csv_data(train_data_path)
y_test, x_test, ids_test = load_csv_data(test_data_path)

x_train_s,mean,std = standardize(x_train)

x_test_s, test_mean, test_std = standardize(x_test)

In [2]:
initial_gamma = 0.000000035
m = x_train.shape[1]
initial_w = np.array([0] * m)
degree = 2
max_iters = 22

def reg_logistic_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    loss, gradient = reg_logistic_regression_cost_grad(y, tx, w, lambda_)
    w = w - gamma * gradient
    return loss, w
  
def reg_logistic_regression(y, tx, lambda_,initial_w, max_iters, gamma):
    for i in range(max_iters):
        for minibatch_y, minibatch_tx in batch_iter(y, tx, 1): # O(1) runtime
            loss, initial_w = reg_logistic_gradient (minibatch_y, minibatch_tx, initial_w, gamma, lambda_)
    
    return initial_w, loss
  
def cross_validation_reg(y, x, k_indices, k, lambda_, degree, w, max_iters, gamma):
    """return the loss of ridge regression."""
    # get k'th subgroup in test, others in train
    te_indice = k_indices[k]
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    y_te = y[te_indice]
    y_tr = y[tr_indice]
    x_te = x[te_indice]
    x_tr = x[tr_indice]
    w, _ = reg_logistic_regression(y_tr, x_tr, lambda_, initial_w, max_iters, gamma)
    # calculate the loss for train and test data
    loss_tr, _ = reg_logistic_regression_cost_grad(y_tr, x_tr, w, lambda_)
    loss_te, _ = reg_logistic_regression_cost_grad(y_te, x_te, w, lambda_)
    return loss_tr, w, loss_te

def cross_validation_demo_reg():
    seed = 12
    k_fold = 10
    lambdas = np.logspace(-4, 0, 30)
    # split data in k fold
    k_indices = build_k_indices(y_train, k_fold, seed)
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    # cross validation
    loss_tr = 0
    loss_te = 0
    w = np.array([0] * x_train.shape[1])
    dict_lambda_weight = {}
    for lambda_ in lambdas:
        rmse_tr_tmp = []
        rmse_te_tmp = []
        for k in range(k_fold):
            loss_tr, weight, loss_te = cross_validation_reg(y_train, x_train_s, k_indices, k, lambda_, degree, initial_w, max_iters, initial_gamma)
            if lambda_ in dict_lambda_weight:
                if dict_lambda_weight[lambda_][0] > loss_te:
                    dict_lambda_weight[lambda_] = (loss_te, weight)
            else: 
                dict_lambda_weight[lambda_] = (loss_te, weight)
            rmse_tr_tmp.append(loss_tr)
            rmse_te_tmp.append(loss_te)
            
        rmse_tr.append(np.mean(rmse_tr_tmp))
        rmse_te.append(np.mean(rmse_te_tmp))
        
    ind_lambda_opt = np.argmin(rmse_te)
    best_lambda = lambdas[ind_lambda_opt]
    best_rmse = rmse_te[ind_lambda_opt]
    best_weight = dict_lambda_weight[best_lambda][1]
    cross_validation_visualization(lambdas, rmse_tr, rmse_te)
    return best_weight, best_rmse
  
optimal_weight, cost = cross_validation_demo_reg()
x_test = build_poly(x_test, degree)
y_pred = predict_labels(optimal_weight, x_test)
#y_pred

output = accuracy(y_pred, y_test)
print(output)
print(str(datetime.now()))

create_csv_submission(ids_test, y_pred,'reg_logistic_regression' + str(datetime.now()) + '.csv') 

NameError: name 'x_train' is not defined